# DeepTesla Jr.
## Extracción de frames de los videos

In [ ]:

from __future__ import print_function

import numpy as np
import cv2
import os

base_path = "J:/Datasets/deeptesla/"
validation_set = [9, 10]
frames_writed = 0

for video_id in xrange(1, 11):
    video_filename = "epoch%02d_front.mkv" % video_id
    video_path = os.path.join(base_path, video_filename)
    assert os.path.isfile(video_path)

    # Create capturator
    cap = cv2.VideoCapture(video_path)
    print("Capturing from ", video_path)

    # Capture first frame
    ret, frame = cap.read()
    assert ret

    # Print shape
    shape = frame.shape
    print(shape)

    while (ret):
        # Crop frame 
        img = frame[int(shape[0]/2):shape[0], 0:shape[1]]
        # Resize the image
        img = cv2.resize(img, (128, 64), interpolation=cv2.INTER_AREA)
        # Resize the image sized as a 4D array
        img = np.resize(img, (64, 128, 3))
        
        # Save the image
        if video_id in validation_set:
            image_path = os.path.join(base_path, "validation/frame%05d.jpg" % frames_writed)
        else:
            image_path = os.path.join(base_path, "training/frame%05d.jpg" % frames_writed)
        cv2.imwrite(image_path, img)
        frames_writed += 1
        if frames_writed % 100 == 0:
            print("Writed %d images..." % frames_writed)

        # Display
        #cv2.imshow('frame', frame)
        #cv2.imshow('img', img)

        # User exit?
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Capture next frame
        ret, frame = cap.read()

    # Release the capture
    cap.release()

cv2.destroyAllWindows()


## Entrenamiento de la CNN

In [ ]:
from __future__ import print_function

import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from PIL import Image
import glob
import os
import pickle
import numpy as np
#import scipy

batch_size = 128
epochs = 6
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'deeptesla_trained_model.h5'

filelist = glob.glob("/mnt/j/Datasets/deeptesla/training/*.jpg")
x_train = np.array([np.array(Image.open(fname)) for fname in filelist])
filelist = glob.glob("/mnt/j/Datasets/deeptesla/validation/*.jpg")
x_test = np.array([np.array(Image.open(fname)) for fname in filelist])

y_train = np.genfromtxt("/mnt/j/Datasets/deeptesla/training.csv", delimiter = ',')
y_test = np.genfromtxt("/mnt/j/Datasets/deeptesla/validation.csv", delimiter = ',')

model = Sequential()
model.add(Conv2D(8, (3, 3), padding = 'same',
                 input_shape = x_train[0].shape))
model.add(Activation('relu'))

model.add(Conv2D(8, (3, 3),
                 kernel_initializer = 'glorot_normal'))
model.add(Activation('relu'))

model.add(Conv2D(8, (3, 3),
                 kernel_initializer = 'glorot_normal'))
model.add(Activation('relu'))

#model.add(MaxPooling2D(pool_size=(2, 2)))
#model.add(Dropout(0.25))

#model.add(Conv2D(64, (3, 3), padding='same'))
#model.add(Activation('relu'))

model.add(MaxPooling2D(pool_size = (2, 2)))
model.add(Dropout(0.5))

model.add(Flatten())

model.add(Dense(256))
model.add(Activation('tanh'))

model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('linear'))

model.compile(loss = 'mean_squared_error',
              optimizer = 'adadelta')

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

model.fit(x_train, y_train,
          batch_size = batch_size,
          epochs = epochs,
          validation_data = (x_test, y_test),
          shuffle = False,
          verbose = 1)

# Save model and weights
#if not os.path.isdir(save_dir):
#    os.makedirs(save_dir)

#model_path = os.path.join(save_dir, model_name)
#model.save(model_path)

#print('Saved trained model at %s ' % model_path)


Train on 21600 samples, validate on 5400 samples
Epoch 1/6
14592/21600 [===================>..........] - ETA: 105s - loss: 16.6062